In [ ]:
# импорт бибилиотек
import pydicom
import os
from collections import defaultdict
from pathlib import Path
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
from pydicom import Dataset
from pydicom.pixel_data_handlers.util import apply_modality_lut, apply_voi_lut, apply_windowing

# Старался использовать все методы и функци из практики для облегчения проверки.

In [ ]:
# задаём параметры по умолчанию 
PATH_TO_DATA = Path('./study')
RANDOM_FILE = PATH_TO_DATA / "IOP.25"

In [ ]:
# Получаем информацию о исслдоевании
ds = pydicom.dcmread(RANDOM_FILE)
ds

In [ ]:
# прочитаем все файлы
files = []
for file in os.listdir(PATH_TO_DATA):
    try:
        files.append(pydicom.dcmread(PATH_TO_DATA / file))
    except:
        pass

In [ ]:
series = defaultdict(list) # Разделим полученные файлы по сериям
for file in files:
    series[file.SeriesInstanceUID].append(file)
for k,i in series.items():
    print(f"series uid {k} with {len(i)} instances")

In [ ]:
# Будем использовать самую длинную серию
biggest_series = sorted(series.items(), key=lambda x:len(x[1]), reverse=True)[0][1]
plt.imshow(biggest_series[0].pixel_array, cmap="gray")


# Критерий 1,4,5

In [ ]:
def dicom_to_raster(dataset: Dataset) -> np.ndarray:
    img = apply_voi_lut(apply_modality_lut(dataset.pixel_array, dataset), dataset)
    image = img.astype(np.float32)
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    return (image * 255).astype("uint8")

In [ ]:
# переведем всю серию в пнг и соберем в единый массив
biggest_series = sorted(biggest_series, key=lambda x: x.SliceLocation)
series_arr = [dicom_to_raster(instance) for instance in biggest_series]
# Сохраним снимки в папку data1
i = 0 
for im in series_arr:
    path = f"data1/{i}.png"
    cv.imwrite(path, im)
    i+=1 






In [ ]:
#сохраним снимки с изменённой контрастностью в папке data2
i = 0 
for im in series_arr:
    path = f"data2/{i}.png"
    new_img = im.copy()
    new_img[im<200] = 0
    im = cv.cvtColor(new_img, cv.COLOR_GRAY2RGB)
    cv.imwrite(path, im)
    i+=1

# Критерий 2

In [ ]:
# Переводим массив со снимками в нампаевский массив
from scipy import ndimage
img3d = np.array(series_arr)
# поворачиваем "кубик", сохраняем в саггитальной плоскости
img3d = ndimage.rotate(img3d, 90, axes=(2, 0),reshape=False)
# Сохраним снимки в папку data1
i = 0 
for im in img3d:
    path = f"data3/{i}.png"
    im =  cv.rotate(im, cv.ROTATE_90_COUNTERCLOCKWISE)
    cv.imwrite(path, im)
    i+=1 
# Переводим массив со снимками в нампаевский массив
from scipy import ndimage
img3d = np.array(series_arr)
# поворачиваем "кубик", сохраняем во фронтальной плоскости
img3d = ndimage.rotate(img3d, 90,reshape=False)
# Сохраним снимки в папку data1
i = 0 
for im in img3d:
    path = f"data4/{i}.png"
    im = cv.rotate(im, cv.ROTATE_180)
    cv.imwrite(path, im)
    i+=1 

# Спасибо за проверку!